# REQUIRED PACKAGE INSTALL

In [10]:
# pip install function
import subprocess
import sys

def install(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except:
        print("No such package")

In [11]:
# pip install packages
install('pytesseract')
install('pdf2image')
install('PyPDF2')
install('PyMuPDF')

In [1]:
import pytesseract
from pdf2image import convert_from_path
import PyPDF2
import io
import fitz # imports the pymupdf library

In [13]:
# uncomment the code below if you don't have tesseract added to path
# pytesseract.pytesseract.tesseract_cmd = r"Tesseract-OCR/tesseract.exe"

# INPUT LAYER

In [11]:
# pdf_doc = r'files/woodBoy.pdf'
pdf_doc = r'files/Brabender_P1.pdf'

# PARSING LAYER

In [3]:
# imagePDF_to_text function
def imagePDF_to_text(file):
    # poppler path
    poppler_path=r"Release-24.02.0-0/poppler-24.02.0/Library/bin"
    # convert to image
    images = convert_from_path(file, poppler_path=poppler_path)
    
    pdf_writer = PyPDF2.PdfWriter()
    page_text = ''
    for image in images:
        page = pytesseract.image_to_pdf_or_hocr(image, extension='pdf')
        pdf = PyPDF2.PdfReader(io.BytesIO(page))

        page_text += (pdf.pages[0]).extract_text()
        pdf_writer.add_page(pdf.pages[0])
    
    return page_text

In [4]:
# PDF_to_text function
def PDF_to_text(file):
  try:
    # works only for text-based pdf
    doc = fitz.open(file) # open a document
    for page in doc: # iterate the document pages
      text = page.get_text() # get plain text encoded as UTF-8
      if not text.strip():
        raise Exception('PDF is a scanned document')
    print('Done!')
    return text
  except Exception as e:
    print(f"Error while processing PDF: {e}")
    print('Running OCR engine...')
    text = imagePDF_to_text(file)
    print('Done!\n')
    return text

In [12]:
text = PDF_to_text(pdf_doc)
# print(text)

Error while processing PDF: PDF is a scanned document
Running OCR engine...
Done!



In [18]:
# export the searchable PDF to searchable.pdf
# out_file = f'{pdf_doc[:-4]}_searchable.pdf'
# with open(out_file, "wb") as f:
#     pdf_writer.write(f)

In [20]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'score': 0.03794672340154648,
 'start': 0,
 'end': 21,
 'answer': "Chocolate's Adventure"}

In [27]:
# pipe(
#     question="what did he discover in the forest?",
#     context=text,
# )


{'score': 0.2770726680755615,
 'start': 529,
 'end': 581,
 'answer': 'secrets that lay hidden beneath the canopy of leaves'}

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

c:\Users\spidy\anaconda3\envs\dmml\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\spidy\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:

# Tokenize and generate summary
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=1500, min_length=400, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print(summary)


Brabender® and other brands not specially marked are registered trademarks of Brabender GmbH & Co. KG. e Each Brabender device is tested as an individual consumer with the recommended RCDs and the real leakage current is documented. e Brabender recommends the RCD type DFS4 B SK made by Doepke. a structurally identical RCD with the type designation RCCB2 is available from Messrs. EPA. ® Do not connect more than...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................